In [1]:

import cv2
import numpy as np
import os
from tqdm import tqdm

def show_image(image, title="Image", scale=5):
    h, w = image.shape
    resized_image = cv2.resize(image, (w * scale, h * scale), interpolation=cv2.INTER_NEAREST)
    cv2.imshow(title, resized_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# load data

final_data = "C:/Users/HUY/Desktop/my_project/final_data"
synthetic_path = "C:/Users/HUY/Desktop/my_project/synthetic_data"

# tải dữ liệu dựa theo góc của dáng đi
def load_view(folder_paths, view):
    train, test = [], []
    train_view = []
    test_view = []
    for folder in (os.listdir(folder_paths)):
        if folder == "train":
            tr_labels = []
            for img in os.listdir(f"{folder_paths}/{folder}"):
                if view in img:
                    im = np.array(cv2.imread(f"{folder_paths}/{folder}/{img}", cv2.IMREAD_GRAYSCALE))
                    tr_labels.append(img[0:3])
                    train.append(im)
                    train_view.append(view)
        if folder == "test":
            te_labels = []
            for img in os.listdir(f"{folder_paths}/{folder}"):
                if view in img:
                    im = np.array(cv2.imread(f"{folder_paths}/{folder}/{img}", cv2.IMREAD_GRAYSCALE))
                    te_labels.append(img[0:3])
                    test.append(im)
                    test_view.append(view)
    # print(f"Hoàn thành tải dữ liệu lên từ {folder_paths} với góc {view} độ")
    return train, test, tr_labels, te_labels, train_view, test_view


# tải dữ liệu dáng đi với tất cả các góc
def load_data(view = "crossview"):
    
    views = ["000","018","036","054","072","090","108","126","144","162","180"]
    x_train, y_train = [], []
    x_test, y_test = [], []
    train_view, test_view = [], []
    
    if view in views:
        views = [view]
    
    for view in tqdm(views):
        x_rtrain_view, x_rtest_view, y_rtrain_view, y_rtest_view, rtrain_view, rtest_view = load_view(final_data, view)
        x_strain_view, x_stest_view, y_strain_view, y_stest_view, strain_view, stest_view = load_view(synthetic_path, view)
        
        x_train = x_train + x_rtrain_view + x_strain_view
        y_train = y_train + y_rtrain_view + y_strain_view
        x_test = x_test + x_rtest_view + x_stest_view
        y_test = y_test + y_rtest_view + y_stest_view
        train_view = train_view + rtrain_view + strain_view
        test_view = test_view + rtest_view + stest_view
        
    print(f"✅ Kích thước tập train và nhãn: {len(x_train)} và {len(y_train)}")
    print(f"✅ Kích thước tập test và nhãn: {len(x_test)} và {len(y_test)}")
    return x_train, y_train, x_test, y_test, train_view, test_view

In [2]:
x_train, y_train, x_test, y_test, train_view, test_view = load_data(view = "crossview")

100%|██████████| 11/11 [00:13<00:00,  1.24s/it]

✅ Kích thước tập train và nhãn: 8502 và 8502
✅ Kích thước tập test và nhãn: 5668 và 5668


In [3]:
# chuyển nhãn dán dữ thành mã hóa one-hot để đưa vào mô hình CNN

from sklearn.preprocessing import LabelEncoder

import numpy as np

def to_categorical(y, num_classes=None):
    y = np.array(y, dtype=int)
    if num_classes is None:
        num_classes = np.max(y) + 1
    one_hot = np.zeros((len(y), num_classes))
    one_hot[np.arange(len(y)), y] = 1
    return one_hot


all_labels = np.concatenate([y_train, y_test])

label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

y_train_int = label_encoder.transform(y_train)
y_test_int = label_encoder.transform(y_test)

# one-hot encoding
num_classes = len(label_encoder.classes_)
y_train_encoded = to_categorical(y_train_int, num_classes)
y_test_encoded = to_categorical(y_test_int, num_classes)

# OKEOKE

In [4]:
x_train = np.array(x_train)
x_test = np.array(x_test)

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# OKE

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential()

# Convolutional layer 1
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 44, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional layer 2
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Convolutional layer 3
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten và fully connected layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # tránh overfitting
model.add(Dense(10, activation='softmax'))  # 15 class output

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 62, 42, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 62, 42, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 31, 21, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 29, 19, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 29, 19, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 14, 9, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 12, 7, 128)     │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 12, 7, 128)     │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 6, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 389,898 (1.49 MB)

 Trainable params: 389,450 (1.49 MB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Huấn luyện mô hình
history = model.fit(
    x_train, y_train_encoded,
    validation_data=(x_test, y_test_encoded),
    epochs=15, # FIX HERE
    batch_size=32,
    verbose=1
)

Epoch 1/30
266/266 ━━━━━━━━━━━━━━━━━━━━ 45s 158ms/step - accuracy: 0.1993 - loss: 2.3821 - val_accuracy: 0.1570 - val_loss: 10.8706
Epoch 2/30
266/266 ━━━━━━━━━━━━━━━━━━━━ 45s 170ms/step - accuracy: 0.4051 - loss: 1.5025 - val_accuracy: 0.3294 - val_loss: 1.5921
Epoch 3/30
266/266 ━━━━━━━━━━━━━━━━━━━━ 51s 192ms/step - accuracy: 0.5823 - loss: 1.0586 - val_accuracy: 0.8237 - val_loss: 0.6784
Epoch 4/30
266/266 ━━━━━━━━━━━━━━━━━━━━ 44s 165ms/step - accuracy: 0.7040 - loss: 0.7703 - val_accuracy: 0.8731 - val_loss: 0.4347
Epoch 5/30
266/266 ━━━━━━━━━━━━━━━━━━━━ 44s 165ms/step - accuracy: 0.8196 - loss: 0.4781 - val_accuracy: 0.8754 - val_loss: 0.4025
Epoch 6/30
266/266 ━━━━━━━━━━━━━━━━━━━━ 49s 183ms/step - accuracy: 0.8865 - loss: 0.3079 - val_accuracy: 0.9116 - val_loss: 0.4396
Epoch 7/30
266/266 ━━━━━━━━━━━━━━━━━━━━ 59s 221ms/step - accuracy: 0.9166 - loss: 0.2161 - val_accuracy: 0.9345 - val_loss: 0.2762
Epoch 8/30
266/266 ━━━━━━━━━━━━━━━━━━━━ 42s 157ms/step - accuracy: 0.9320 - loss: 

In [9]:
# Đánh giá mô hình
test_loss, test_acc = model.evaluate(x_test, y_test_encoded, verbose=0)
print(f"\n✅ Test accuracy: {test_acc:.4f} - Test loss: {test_loss:.4f}")


✅ Test accuracy: 0.9670 - Test loss: 0.2226


In [10]:
model.save("gaitEnergy_cnn.h5")